<a href="https://colab.research.google.com/github/UmarNauruzov/Neural_network/blob/main/%D0%9E%D0%B1%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D0%BA%D0%B0_%D1%82%D0%B5%D0%BA%D1%81%D1%82%D0%B0_%D1%82%D0%B2%D0%B8%D1%82%D0%BE%D0%B2_%D1%81_%D0%BF%D0%BE%D0%BC%D0%BE%D1%89%D1%8C%D1%8E_%D0%BD%D0%B5%D0%B9%D1%80%D0%BE%D0%BD%D0%BD%D1%8B%D1%85_%D1%81%D0%B5%D1%82%D0%B5%D0%B9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Обработка текстов нейронными сетями: классификация позитивных и неготивных отзывов

##Импорт библиотек

In [ ]:
# Работа с массивами данных
import numpy as np 

# Работа с таблицами
import pandas as pd

# Отрисовка графиков
import matplotlib.pyplot as plt

# Функции-утилиты для работы с категориальными данными
from tensorflow.keras import utils

# Класс для конструирования последовательной модели нейронной сети
from tensorflow.keras.models import Sequential

# Основные слои
from tensorflow.keras.layers import Dense, Dropout, SpatialDropout1D, BatchNormalization, Embedding, Flatten, Activation 

# Токенизатор для преобразование текстов в последовательности
from tensorflow.keras.preprocessing.text import Tokenizer

# Заполнение последовательностей до определенной длины
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Матрица ошибок классификатора
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# Кодирование тестовых меток
from sklearn.preprocessing import LabelEncoder

# Разбиение на тренировочную и тестовую выборки
from sklearn.model_selection import train_test_split

# Загрузка датасетов из облака google
import gdown

# Отрисовка графиков
import matplotlib.pyplot as plt

%matplotlib inline

##Загрузка датасета и подготовка данных

In [ ]:
!gdown --id 1GujrcFzRdo3E7UtUkcrljzDS9czBBy3s
!gdown --id 1vvm-PrV0r2wuGbYYovZSuReYOXpu0JRK

import os, shutil
if not os.path.exists('/content/data'):
  os.makedirs('/content/data')
shutil.move('train.csv', "/content/data")

Библиотека Pandas поможет прочесть данные в виде таблицы, чтобы работа с ними стала
удобнее.

Сохраните данные в переменную **train**.

Pandas инициирована как `pd`, а `read_csv()` – функция чтения данных табличного
формата.

In [ ]:
train = pd.read_csv('/content/data/train.csv', delimiter=',', index_col='id')
test = pd.read_csv('/content/data/Soc_Net_Task_2_test_5.csv', delimiter=',', index_col='id')
test

,text
id,
214980,RT fcdk1927 этот твит посвящается всем тем кт...
220887,kryakozya kristinkashurda да это хуйня все
215971,RT zyhihogogufi он всё еще держит меня вконтак...
214091,yozhum айпод нано 7 просто нам в сервисном заг...
217379,NikitaLalko я теперь НИЧЕГО не понимаю
...,...
205768,рыдалъ рыдалъ облако тупой фигни InstantFollo...
221397,Eminem Mockingbird\nhttptcoJZnCcGe4LB с помощ...
216347,despairscorpion все хотят быть плюшками


Функция `shape()` сообщает, что в таблице 181467 строк (самих твитов) и 3
столбца, чтобы удобно упорядочить все данные по разделам.

In [ ]:
# Вывод размерности таблицы
print(train.shape)

(181467, 2)


Деление данных по
столбцам, соответственно, это id сообщения, текст сообщения(твита) и его
класс(позитивный или неготивный). Это вы увидите, отобразив первые 5 строк базы с помощью
`head()`:

In [ ]:
# Вывод первых строк таблицы
train.head()                                 

,text,class
id,,
0,VitaliaPerilova у тебя классные очкиони тебя о...,1
1,Никогда не подумала бы что самое приятное на с...,1
2,KlimovaElena о дааа вот это я понимаю как никт...,0
3,Говорят еле хожу никого не вижу будто живой ме...,0
4,то чувство когда понимаешь что завтра можно вы...,1


Функция `isnull()` считает все пустые места в таблице с данными, a `sum()` помогает
сложить их вместе. 

Посмотрите краткую статистику по пропускам:

In [ ]:
# Проверка, есть ли пропуски в данных
print(train.isnull().sum())

text     0
class    0
dtype: int64


Пропуски можно заполнить одинаковым сообщением с помощью `fillna()`, но у нас пустых данных нет:

In [ ]:
# Есть пропуски. Замена всех пустых значений на 'нет данных'
train.fillna('нет данных', inplace=True)

Если бы вы хотели получить наглядную информацию об объемах данных по каждой
категории(классу), то написали бы для этого простую функцию, как в примере ниже.

Имя функции *data_summary* с английского переводится как «обобщенная информация о
данных». Внутри функции всего две строки кода:
1. С циклом `for`, который проходит по всем категориям обращений граждан (в
функции категории сохранены как **class_labels**, а ниже виден исходный
доступ к ним через `train['class']`).
2. Форматированная строка с выводом информации по категориям, где:

    * **cls** – имя для каждой ячейки с названием категории;

    * `train[train["class"]` – доступ ко всему столбцу категорий целиком;

    * добавление `.shape[0]` в конце – доступ к ячейкам построчно, т.е. по
отдельности.

Протестируйте функцию и изучите информацию о количестве сообщений по отдельно
взятой категории:

In [ ]:
# Функция вывода сводки по распределению данных
def data_summary(train, class_labels):
    for cls in class_labels:
        print(f'Количество записей класса {cls}: {train[train["class"] == cls].shape[0]}')   

Метод `unique()` для базы данных за вас подсчитывает количество сообраний по каждой
категории. 

In [ ]:
# Сводка по распределению данных
data_summary(train, train["class"].unique())

Количество записей класса 1: 91938
Количество записей класса 0: 89529


###Преобразование текстовых данных в числовые и векторные представления для обучения нейросети



Если все сообщения – это один строковый образец текста, то надо выделить часть
этого текста по частоте повторений слов. Это и делает токенизатор в Keras.
1. Выделите лимит в 10 000 слов, сохраните его в **VOCAB_SIZE**.
2. `TRAIN_TEST_RATIO = 0.2` буквально означает, что на проверку точности НС
будет выделено 20% от обучающих примеров.

Учитывайте, что и **VOCAB_SIZE**, и **TRAIN_TEST_RATIO** – это просто имена
переменных, придуманные для удобства.

In [ ]:
# Задание параметров преобразования
VOCAB_SIZE       = 10000                  # Объем словаря для токенизатора
TRAIN_TEST_RATIO = 0.2                    # Доля проверочной выборки в основном наборе

**Токенизация и преобразование в последовательности**

В этой части вы поработаете с данными в табличном формате, но окончательный вид
данных для обработки – текст. Со временем вы научитесь замечать, к какому типу
принадлежат данные. При обработке текста сначала объединяют весь набор
тренировочных данных в один неразрывный текст.

Сохраните обобщенный текст всех сообщений в переменной **text_data_train**.
Команда `train['text']` сразу предоставит доступ ко всему столбцу с сообщениями. А
`tolist()` позволит получить списки сообщений, собранные в одно целое.

Тот же набор действий используют для переменной **class_data**, но здесь идет работа
уже с категориями сообщений. Обучаясь на этих данных, НС узнает, как
правильно сопоставить сообщение с его категорией и точно определить ответ. Здесь вы
получите доступ к столбцу категорий `train['class']`, и так же сохраните их все в список
`tolist()`. Тоже самое делаем и для тестовых данных

In [ ]:
# Извлечение всех записей-текстов обращений и их меток классов
text_data_train = train["text"].tolist()
class_data_train = train["class"].tolist()
text_data_test = test["text"].tolist()
# text_data_test 

Tokenizer составит на основе текста сообщений словарь
частотности с ограничением до индекса 10 000 (значение **VOCAB_SIZE**). Тогда каждое
слово получит свой индекс в зависимости от частоты его повторений, а до этого из текста
будут убраны все символы, все заглавные буквы станут строчными и будут разделены
лишь пробелами. Как вы уже знаете, все слова, не попавшие в диапазон **VOCAB_SIZE**
будут признаны неизвестными словами и получат такое же название и индекс 1.

Сам словарь формируется для токенайзера функцией `fit_on_texts()`.

Функция `word_index_items()` поможет образовать тот же словарь в виде пар «слово :
индекс частоты повторений слова».

In [ ]:
# Токенизация и построение частотного словаря по обучающим текстам
# Используется встроенный в Keras токенизатор для разбиения текста и построения частотного словаря
tokenizer = Tokenizer(num_words=VOCAB_SIZE, filters='!"#$%&()*+,-–—./…:;<=>?@[\\]^_`{|}~«»\t\n\xa0\ufeff', lower=True, split=' ', oov_token='неизвестное_слово', char_level=False)

# Использованы параметры:
# num_words   - объем словаря
# filters     - убираемые из текста ненужные символы
# lower       - приведение слов к нижнему регистру
# split       - разделитель слов
# char_level  - указание разделять по словам, а не по единичным символам
# oov_token   - токен для слов, которые не вошли в словарь

# Построение частотного словаря по обучающим текстам
tokenizer.fit_on_texts(text_data_train)

# Построение словаря в виде пар слово - индекс
items = list(tokenizer.word_index.items())

# tokenizer.fit_on_texts(text_data_var)
# items1 = list(tokenizer.word_index.items())

tokenizer.fit_on_texts(text_data_test)

items2 = list(tokenizer.word_index.items())

В библиотеке Sklearn есть очень полезный инструмент *LabelEncoder*. Его используют в
тех случаях, когда работают с категориальными данными. В вашей таблице есть категорий сообщений - «Позитивный» или "Неготивный". НС будет выявлять принадлежность сообращения к
одной из таких категорий.

*LabelEncoder* преобразует весь список категорий текстового типа (`class_data_train`) в список
индексов от 0 и до последней категории по счету. Если какая-либо категория встречается
повторно, то она все равно сохраняет тот индекс, что первоначально ей присвоен. Так,
сколько раз вы бы не встретили категорию «Позитивный», если ее индекс 1, то он так и
останется 1.


In [ ]:
# Кодирование меток классов индексами (числами)
encoder = LabelEncoder()
class_labels = encoder.fit_transform(class_data_train)

CLASS_LIST = encoder.classes_
CLASS_COUNT = len(CLASS_LIST)

print(f'Размер словаря: {len(items)}')
print(f'Список классов: {CLASS_LIST}')
print(f'Всего классов: {CLASS_COUNT}')
print(f'Форма выходных данных: {class_labels.shape}')
print(f'Пример числовых меток классов: {class_labels[:10]}')

Размер словаря: 304899
Список классов: [0 1]
Всего классов: 2
Форма выходных данных: (181467,)
Пример числовых меток классов: [1 1 0 0 1 1 0 0 1 1]


Словарь частотности повторений уже составлен.

Используйте функцию токенайзера `texts_to_sequences()` и получите текст для
обучения в виде одних только индексов (чисел).

Переменная **x_data** задана для подготовки данных к самому обучению, потому же
данные внутри нее сразу преобразуются в numpy-массив через `np.array()`:

Будет здорово, если вы освоите материал занятия и выполните такие же задачи
самостоятельно! Пусть это будет вашей мотивацией.

В блоке кода ниже текст обращений также преобразуется в матрицу
(`texts_to_matrix`) из значений 0 и 1 для изученного ранее способа *Bag Of Words*. Эти
данные сохраняются в переменной **x_data_01**.

Вы помните, что верные ответы (категории сообщений) были переведены *LabelEncoder* в
формат индексов. Но нейронные сети хорошо работают именно с нормализованными
данными. Формат от 0 до 1 – лучшая нормализация. Однако и ряд индексов категорий –
это не полноценный текст, а просто слова и словосочетания (метки классов). Для
преобразования категорий ответов для подачи в сеть чаще всего используют метод
`utils.to_categorical`. Он дает вектор из нулей и одной единицы (положение
верного ответа) для каждой категории в отдельности.

In [ ]:
# Преобразование входных текстов в последовательности индексов (для архитектур с embedding)
# Применено приведение к массиву объектов для дальнейшего разделения на выборки
x_data = np.array(tokenizer.texts_to_sequences(text_data_train), dtype=object)
# Вывод формы и примера данных
print(x_data.shape) 
print(x_data[0][:20])

# Преобразование входных текстов в разреженную матрицу из векторов Bag of Words
x_data_01 = tokenizer.texts_to_matrix(text_data_train)
# Вывод формы и примера данных
print(x_data_01.shape) 
print(x_data_01[0, :20])

# Преобразование меток класса к векторам one hot encoding
y_data = utils.to_categorical(class_labels, CLASS_COUNT)
# Вывод формы и примера данных
print(y_data.shape)
print(y_data[0])

(181467,)
[1, 11, 34, 1901, 1, 34, 36, 1, 1, 52, 93, 7925]
(181467, 30000)
[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
(181467, 2)
[0. 1.]


**Создание обучающей и тестовой выборок**

На данный момент вы располагаете **x_data** – общим массивом чисел для обучения.

Этот массив необходимо поделить на обучающую выборку и тестовую, ведь вы хотите
тестировать точность сети на независимых данных! Метод `train_test_split()`
поможет провести границу деления, результат которого для границы обучающей выборки
сохраняют переменные **idx_train** и **idx_test** – для тестовой.

В параметрах метода:
1. `range(len( ))` для **x_data** – диапазон длины всего массива для обучения.
2. `stratify = y_data` – сопоставление тренировочных и тестовых данных с
ответами к ним.

3. `test_size = TRAIN_TEST_RATIO` – то самое значение деления
(`TRAIN_TEST_RATIO = 0.2`) общего массива на 80% (тренировочная выборка) к
20% (тестовая выборка).

Поместите **idx_train**, **idx_test** в диапазон **x_data** (`x_data[idx_train],
x_data[idx_test]`). Так вы получите уже финальные выборки. Их можно подать на
обучение сети. То же самое сделайте для **x_data_01**, чтобы получить выборки для
способа *Bag Of Words*.

В конце важно проверить размерности каждой выборки, чтобы правильно задать
гиперпараметры обучения НС. Это можно сделать с помощью `shape()`

In [ ]:
# Получение индексов разделения основного набора на обучающую и тестовую выборки
idx_train, idx_test = train_test_split(list(range(len(x_data))),
                                       stratify=y_data,
                                       test_size=TRAIN_TEST_RATIO)

# Разделение в соответствии с полученными индексами
x_train, x_test = x_data[idx_train], x_data[idx_test]
x_train_01, x_test_01 = x_data_01[idx_train], x_data_01[idx_test]
y_train, y_test = y_data[idx_train], y_data[idx_test]

# Проверка результата
print(x_train.shape, x_test.shape)
print(x_train_01.shape, x_test_01.shape)
print(y_train.shape, y_test.shape)

###Сервисные функции


Ниже представленные функции собирают параметры обучения модели воедино (компиляция), оценивают
точность уже обученной сети и выводят графики процесса обучения и точности. Две
первые функции последовательно вызываются в третьей –
`compile_train_eval_model()`. Именно ей вы и воспользуетесь.

In [ ]:
# Функция компиляции и обучения модели нейронной сети
def compile_train_model(model, 
                        x_train,
                        y_train,
                        x_val,
                        y_val,
                        optimizer='adam',
                        epochs=50,
                        batch_size=128,
                        figsize=(20, 5)):

    # Компиляция модели
    model.compile(optimizer=optimizer, 
                  loss='categorical_crossentropy', 
                  metrics=['accuracy'])

    # Вывод сводки
    model.summary()

    # Обучение модели с заданными параметрами
    history = model.fit(x_train,
                        y_train,
                        epochs=epochs,
                        batch_size=batch_size,
                        validation_data=(x_val, y_val))

    # Вывод графиков точности и ошибки
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=figsize)
    fig.suptitle('График процесса обучения модели')
    ax1.plot(history.history['accuracy'], 
               label='Доля верных ответов на обучающем наборе')
    ax1.plot(history.history['val_accuracy'], 
               label='Доля верных ответов на проверочном наборе')
    ax1.xaxis.get_major_locator().set_params(integer=True)
    ax1.set_xlabel('Эпоха обучения')
    ax1.set_ylabel('Доля верных ответов')
    ax1.legend()

    ax2.plot(history.history['loss'], 
               label='Ошибка на обучающем наборе')
    ax2.plot(history.history['val_loss'], 
               label='Ошибка на проверочном наборе')
    ax2.xaxis.get_major_locator().set_params(integer=True)
    ax2.set_xlabel('Эпоха обучения')
    ax2.set_ylabel('Ошибка')
    ax2.legend()
    plt.show()


# Функция вывода результатов оценки модели на заданных данных
def eval_model(model, x, y_true,
               class_labels=[],
               cm_round=3,
               title='',
               figsize=(10, 10)):
    # Вычисление предсказания сети
    y_pred = model.predict(x)
    # Построение матрицы ошибок
    cm = confusion_matrix(np.argmax(y_true, axis=1),
                          np.argmax(y_pred, axis=1),
                          normalize='true')
    # Округление значений матрицы ошибок
    cm = np.around(cm, cm_round)

    # Отрисовка матрицы ошибок
    fig, ax = plt.subplots(figsize=figsize)
    ax.set_title(f'Нейросеть {title}: матрица ошибок нормализованная', fontsize=18)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_labels)
    disp.plot(ax=ax)
    plt.gca().images[-1].colorbar.remove()  # Стирание ненужной цветовой шкалы
    plt.xlabel('Предсказанные классы', fontsize=16)
    plt.ylabel('Верные классы', fontsize=16)
    fig.autofmt_xdate(rotation=45)          # Наклон меток горизонтальной оси при необходимости
    plt.show()    

    print('-'*100)
    print(f'Нейросеть: {title}')

    # Для каждого класса:
    for cls in range(len(class_labels)):
        # Определяется индекс класса с максимальным значением предсказания (уверенности)
        cls_pred = np.argmax(cm[cls])
        # Формируется сообщение о верности или неверности предсказания
        msg = 'ВЕРНО :-)' if cls_pred == cls else 'НЕВЕРНО :-('
        # Выводится текстовая информация о предсказанном классе и значении уверенности
        print('Класс: {:<20} {:3.0f}% сеть отнесла к классу {:<20} - {}'.format(class_labels[cls],
                                                                               100. * cm[cls, cls_pred],
                                                                               class_labels[cls_pred],
                                                                               msg))

    # Средняя точность распознавания определяется как среднее диагональных элементов матрицы ошибок
    print('\nСредняя точность распознавания: {:3.0f}%'.format(100. * cm.diagonal().mean()))


# Совместная функция обучения и оценки модели нейронной сети
def compile_train_eval_model(model, 
                             x_train,
                             y_train,
                             x_test,
                             y_test,
                             class_labels=CLASS_LIST,
                             title='',
                             optimizer='adam',
                             epochs=50,
                             batch_size=128,
                             graph_size=(20, 5),
                             cm_size=(10, 10)):

    # Компиляция и обучение модели на заданных параметрах
    # В качестве проверочных используются тестовые данные
    compile_train_model(model, 
                        x_train, y_train,
                        x_test, y_test,
                        optimizer=optimizer,
                        epochs=epochs,
                        batch_size=batch_size,
                        figsize=graph_size)

    # Вывод результатов оценки работы модели на тестовых данных
    eval_model(model, x_test, y_test, 
               class_labels=class_labels, 
               title=title,
               figsize=cm_size)

### BagOfWords + Dense

Модель – полносвязная НС с применением *Bag Of Words*. Исходя из этого, имя модели **model_text_bow_dense** состоит из 7 слоев, из
которых 4 слоя – полносвязные (Dense). Каждый Dense-слой сети включает по 100 нейронов и функцию активации `relu`. Кроме
слоя выхода там должно получиться число нейронов в соответствии с количеством
определяющихся категорий (2, это число сохранено в **CLASS_COUNT**). Функция
активации тоже отлична (здесь `softmax`).

Скомпилируйте, обучите модель и посмотрите на результат.

In [ ]:
# Последовательная модель
model_text_bow_dense = Sequential()
# Входной полносвязный слой
model_text_bow_dense.add(Dense(100, input_dim=VOCAB_SIZE, activation="relu"))
# Слой регуляризации Dropout
model_text_bow_dense.add(Dropout(0.4))
# Второй полносвязный слой
model_text_bow_dense.add(Dense(100, activation='relu'))
# Слой регуляризации Dropout
model_text_bow_dense.add(Dropout(0.4))
# Третий полносвязный слой
model_text_bow_dense.add(Dense(100, activation='relu'))
# Слой регуляризации Dropout
model_text_bow_dense.add(Dropout(0.4))
# Выходной полносвязный слой
model_text_bow_dense.add(Dense(CLASS_COUNT, activation='softmax'))

# Входные данные подаются в виде векторов bag of words
compile_train_eval_model(model_text_bow_dense,
                         x_train_01, y_train,
                         x_test_01, y_test,
                         epochs=15,
                         batch_size=32,
                         cm_size=(16, 16),
                         class_labels=CLASS_LIST,
                         title='BoW + dense')

Преобразуем тестовые данные для составления прогноза.

In [ ]:
# Преобразование входных текстов в последовательности индексов (для архитектур с embedding)
# Применено приведение к массиву объектов для дальнейшего разделения на выборки
x_test_1 = np.array(tokenizer.texts_to_sequences(text_data_test), dtype=object)
# Вывод формы и примера данных
print(x_test_1.shape) 
print(x_test_1[0][:20])

# Преобразование входных текстов в разреженную матрицу из векторов Bag of Words
x_test_01 = tokenizer.texts_to_matrix(text_data_test)
# Вывод формы и примера данных
print(x_test_01.shape) 
print(x_test_01[0, :20])

Непосредственная подача данных для обученной нейросети.

In [ ]:
# Получение прогнозов модели для новых данных
y_pred = model_text_bow_dense.predict(x_test_01)

# Преобразование прогнозов в бинарный формат
y_pred_binary = np.argmax(y_pred, axis=1)


Сохранения полученного предсказания в txt файл

In [ ]:
# Сохранение прогнозов в текстовый файл через запятую
np.savetxt("predictions4.txt", y_pred, delimiter=",", fmt="%d")